In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission= pd.read_csv('sample_submission.csv')
clf_submission= pd.read_csv('clf_submission.csv')
syn_submission=pd.read_csv('syn_submission.csv')

In [66]:
# difference 칼럼 추가 : 거래 후 잔액  - 거래 전 잔액
train['difference'] = train['Account_balance'] - train['Account_initial_balance']
test['difference'] = test['Account_balance'] - test['Account_initial_balance']


In [67]:
#관련 없는 칼럼 제거
train = train.drop(labels=['ID',
                   'Customer_personal_identifier',
                   'Customer_identification_number',
                   'Account_account_number',
                   'Recipient_Account_Number',
                    'Account_amount_daily_limit',
                    'Account_remaining_amount_daily_limit_exceeded',
                    'IP_Address',
                    'MAC_Address',
                    'Location'],axis=1)

test = test.drop(labels=['ID',
                   'Customer_personal_identifier',
                   'Customer_identification_number',
                   'Account_account_number',
                   'Recipient_Account_Number',
                    'Account_amount_daily_limit',
                    'Account_remaining_amount_daily_limit_exceeded',
                    'IP_Address',
                    'MAC_Address',
                    'Location'],axis=1)



In [68]:
train_target = train['Fraud_Type'].value_counts()
print(train_target)


Fraud_Type
m    118800
a       100
j       100
h       100
k       100
c       100
g       100
i       100
b       100
f       100
d       100
e       100
l       100
Name: count, dtype: int64


In [69]:
# 수치형 데이터 스케일링 수행
#Transaction_num_connection_failure,Number_of_transaction_with_the_account,Transaction_history_with_the_account 제외
#이유 : 데이터 분포가 넓지 않아서 (예시 : 0~ 10 범위)
from sklearn.preprocessing import StandardScaler


train_scaler = train[['Customer_Birthyear',
                    'Account_initial_balance',
                    'Account_balance',
                    'difference',
                    'Account_one_month_max_amount',
                    'Account_one_month_std_dev',
                    'Account_dawn_one_month_max_amount',
                    'Account_dawn_one_month_std_dev',
                    'Transaction_Amount',
                    'Distance']]
test_scaler = test[['Customer_Birthyear',
                    'Account_initial_balance',
                    'Account_balance',
                    'difference',
                    'Account_one_month_max_amount',
                    'Account_one_month_std_dev',
                    'Account_dawn_one_month_max_amount',
                    'Account_dawn_one_month_std_dev',
                    'Transaction_Amount',
                    'Distance']]

scaler = StandardScaler()

scaled_train = scaler.fit_transform(train_scaler)
scaled_test = scaler.fit_transform(test_scaler)

columns=['sc_Customer_Birthyear',
        'sc_Account_initial_balance',
        'sc_Account_balance',
        'sc_difference',
        'sc_Account_one_month_max_amount',
        'sc_Account_one_month_std_dev',
        'sc_Account_dawn_one_month_max_amount',
        'sc_Account_dawn_one_month_std_dev',
        'sc_Transaction_Amount',
        'sc_Distance']
scaled_train = pd.DataFrame(data = scaled_train,columns=columns)
scaled_test = pd.DataFrame(data = scaled_test,columns=columns)


train=train.drop(['Customer_Birthyear',
                    'Account_initial_balance',
                    'Account_balance',
                    'difference',
                    'Account_one_month_max_amount',
                    'Account_one_month_std_dev',
                    'Account_dawn_one_month_max_amount',
                    'Account_dawn_one_month_std_dev',
                    'Transaction_Amount',
                    'Distance'],axis=1)
test=test.drop(['Customer_Birthyear',
                    'Account_initial_balance',
                    'Account_balance',
                    'difference',
                    'Account_one_month_max_amount',
                    'Account_one_month_std_dev',
                    'Account_dawn_one_month_max_amount',
                    'Account_dawn_one_month_std_dev',
                    'Transaction_Amount',
                    'Distance'],axis=1)

train = pd.concat([train,scaled_train],axis = 1)
test = pd.concat([test,scaled_test],axis = 1)

train.head()

,Customer_Gender,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,Customer_rooting_jailbreak_indicator,Customer_mobile_roaming_indicator,Customer_VPN_Indicator,Customer_loan_type,Customer_flag_terminal_malicious_behavior_1,Customer_flag_terminal_malicious_behavior_2,Customer_flag_terminal_malicious_behavior_3,Customer_flag_terminal_malicious_behavior_4,Customer_flag_terminal_malicious_behavior_5,Customer_flag_terminal_malicious_behavior_6,Customer_inquery_atm_limit,Customer_increase_atm_limit,Account_account_type,Account_creation_datetime,Account_indicator_release_limit_excess,Account_indicator_Openbanking,Account_release_suspention,Transaction_Datetime,Channel,Operating_System,Error_Code,Transaction_Failure_Status,Type_General_Automatic,Access_Medium,Transaction_num_connection_failure,Another_Person_Account,Time_difference,Unused_terminal_status,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,sc_Customer_Birthyear,sc_Account_initial_balance,sc_Account_balance,sc_difference,sc_Account_one_month_max_amount,sc_Account_one_month_std_dev,sc_Account_dawn_one_month_max_amount,sc_Account_dawn_one_month_std_dev,sc_Transaction_Amount,sc_Distance
0,male,2003-01-06 18:38:01,B,0,1,0,1,0,0,0,a,0,0,0,0,0,0,0,0,c,2003-01-22 23:38:48,0,1,1,2003-01-25 22:20:34,internet,Others,a,0,general,b,0,1,0 days 02:53:50,1,2003-01-22 23:38:48,2003-01-22 23:38:48,1,1,1,0,0,0,m,2003-01-22 23:38:48,0.183753,-0.132363,-0.198022,-0.071204,-0.690280,-0.619134,-0.383157,-0.362559,-0.259218,2.634305
1,male,2003-01-07 16:40:44,C,0,1,0,0,0,0,0,a,0,0,0,0,0,0,0,0,b,2003-01-19 21:29:08,0,0,1,2003-01-31 02:22:42,Others,Windows,a,0,general,a,0,1,0 days 01:07:33,0,2003-01-21 21:29:08,2003-01-31 00:19:46,0,1,0,0,0,0,m,2003-01-19 21:29:08,-0.830639,-0.126907,-0.273283,-0.134349,-0.220254,0.030757,0.392827,0.369672,-1.129605,-1.209042
2,female,2003-01-11 14:08:36,B,1,1,0,0,0,0,0,c,0,0,0,0,0,0,0,1,d,2003-01-31 07:13:28,1,0,1,2003-01-31 10:20:12,Others,Others,a,0,general,b,0,1,0 days 00:52:59,1,2003-01-31 07:13:28,2003-01-31 07:13:28,0,0,1,1,1,0,m,2003-01-31 07:13:28,0.310552,-0.191918,0.526385,0.541127,-0.667106,-0.619134,-0.383157,-0.362559,0.436538,-0.624172
3,female,2003-01-11 14:08:36,B,1,1,1,0,0,0,0,c,0,0,0,0,0,0,1,1,d,2003-01-31 07:13:28,0,1,1,2003-01-31 13:14:01,Others,Windows,a,0,general,a,2,1,0 days 01:24:05,1,2003-01-31 11:49:56,2003-01-31 07:13:28,1,1,0,0,0,0,m,2003-01-31 07:13:28,0.310552,-0.191918,0.693113,0.673190,-0.648043,-0.582449,-0.383157,-0.362559,0.591804,1.464809
4,female,2003-01-11 14:08:36,B,1,1,1,0,0,0,0,c,0,0,0,1,0,0,1,1,d,2003-01-31 07:13:28,0,1,1,2003-01-31 15:22:26,mobile,Android,a,0,general,b,2,1,0 days 01:43:29,1,2003-01-31 11:49:56,2003-01-31 07:13:28,1,0,0,1,1,0,m,2003-01-31 07:13:28,0.310552,0.611225,-0.222219,-0.571516,-0.648043,-0.562092,-0.383157,-0.362559,-0.260602,0.992205


In [70]:
#범주형 데이터 원-핫 인코딩 수행
# 순서형 데이터 train['Customer_credit_rating'] 먼저 OrdinalEncoder 수행.
from sklearn.preprocessing import OrdinalEncoder

encoder_train = OrdinalEncoder(categories=[['E','D','C','B','A','S']])
train['Customer_credit_rating'] = encoder_train.fit_transform(train[['Customer_credit_rating']]).astype(int)


encoder_test =OrdinalEncoder(categories=[['E','D','C','B','A','S']])
test['Customer_credit_rating'] = encoder_test.fit_transform(test[['Customer_credit_rating']]).astype(int)


In [72]:
# 명목형 데이터 LabelEncoder 수행
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

class MultiColLabelEncoder:
    def __init__(self):
        self.encoder_dict = defaultdict(LabelEncoder)

    def fit_transform(self, X: pd.DataFrame, columns : list):
        if not isinstance(columns, list):
            columns = [columns]

        output = X.copy()
        output[columns] = X[columns].apply(lambda x:self.encoder_dict[x.name].fit_transform(x))

        return output

    def inverse_transform(self, X : pd.DataFrame, columns : list):
        if not isinstance(columns, list):
            columns = [columns]

        if not all(key in self.encoder_dict for key in columns):
            raise KeyError(f'At least one of {columns} is not encoded before')

        output = X.copy()
        try:
            output[columns] = X[columns].apply(lambda x : self.encoder_dict[x.name].inverse_transform(x))
        except ValueError:
            print(f'Need assignment when do "fit_transform" function')
            raise
            
        return output

mcle = MultiColLabelEncoder()
columns_1 = ['Customer_Gender',
           'Customer_loan_type',
           'Account_account_type',
           'Channel',
           'Operating_System',
           'Error_Code',
           'Type_General_Automatic',
           'Access_Medium',
            'Fraud_Type']
columns_2 = ['Customer_Gender',
           'Customer_loan_type',
           'Account_account_type',
           'Channel',
           'Operating_System',
           'Error_Code',
           'Type_General_Automatic',
           'Access_Medium']
transformed_data_train = mcle.fit_transform(train, columns = columns_1)
transformed_data_test = mcle.fit_transform(test, columns = columns_2)





In [73]:
from datetime import datetime

time_columns=['Customer_registration_datetime',
                      'Account_creation_datetime',
                      'Transaction_Datetime',
                      'Last_atm_transaction_datetime',
                      'Last_bank_branch_transaction_datetime',
                      'Transaction_resumed_date'
                      ]

for col in time_columns:
    # 문자열을 datetime으로 변환
    transformed_data_train[col] = pd.to_datetime(transformed_data_train[col])
    transformed_data_test[col] = pd.to_datetime(transformed_data_test[col])
    # datetime을 UNIX 타임스탬프(초)로 변환
    transformed_data_train[col] = transformed_data_train[col].astype('int64') / 10 ** 9
    transformed_data_test[col] = transformed_data_test[col].astype('int64') / 10 ** 9


In [74]:
# Time Difference 를 초 단위로 변경

# train 데이터셋에 대한 변경
transformed_data_train['Time_difference'] = pd.to_timedelta(transformed_data_train['Time_difference'])
transformed_data_train['Time_difference'] = transformed_data_train['Time_difference'].dt.total_seconds()


# test 데이터셋에 대한 변경
transformed_data_test['Time_difference'] = pd.to_timedelta(transformed_data_test['Time_difference'])
transformed_data_test['Time_difference'] = transformed_data_test['Time_difference'].dt.total_seconds()


In [75]:
#데이터셋 분리


from sklearn.model_selection import train_test_split
X = transformed_data_train.drop(['Fraud_Type'],axis=1)
y= transformed_data_train['Fraud_Type']

x_train, x_val, y_train, y_val = train_test_split(X,y , test_size = 0.25,shuffle=True,random_state = 42)

In [76]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_val)

# 모델 평가
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

# 중요 피처 출력(15개만)
importances = rf_model.feature_importances_
feature_importance = pd.Series(importances, index=X.columns).sort_values(ascending=False)[:15]
print("Feature Importances:\n", feature_importance)

Accuracy: 0.9903333333333333
Confusion Matrix:
 [[    0     0     0     0     0     0     0     0     0     0     0     0
     20]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     25]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     27]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     24]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     26]
 [    1     0     0     0     0     0     0     0     0     0     0     0
     19]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     24]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     24]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     21]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     24]
 [    0     0     0     0     0     0     0     0     0     0     0     0
     29]
 [    0     0     0     0     0     0  

C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [80]:
result = rf_model.predict(transformed_data_test)


clf_submission['Fraud_Type'] = result
result_1 = mcle.inverse_transform(clf_submission, columns='Fraud_Type')

result_1.to_csv("clf_submission_1.csv", index=False)
